# Run PEST and Parallel PEST
### This should be super easy ~

In [ ]:
import os
import shutil
import warnings
warnings.filterwarnings("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning) 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt;
import psutil
import sys
import pyemu
import swatp_pst
assert "dependencies" in pyemu.__file__
assert "dependencies" in swatp_pst.__file__

# 01. Specify a path to the model folder

In [ ]:
# path to project directory
prj_dir = "C:\\Users\\seonggyu.park\\Documents\\swatp_gwflow_opt"
# prj_dir = "/Users/seonggyu.park/Documents/projects/jj" #mac

In [ ]:
main_opt_path = os.path.join(prj_dir, 'main_opt')
os.chdir(main_opt_path)

In [ ]:
os.getcwd()

# 02. Initial run for adjusting weights

In [ ]:
# PEST control file name
pst_name = "swatp_dummy.pst"

In [ ]:
# initial run
# pyemu.os_utils.run(f'pestpp-glm.exe {pst_name}' , cwd=".")

## 02-01 Change weights to make all of observation data visible 

In [ ]:
# reweight
pst = pyemu.Pst(pst_name)
pst.phi

In [ ]:
# you can assign any values to balanced phi value for each group
balanced_groups = {grp:1000 for grp in pst.nnz_obs_groups}
pst.adjust_weights(obsgrp_dict=balanced_groups)

In [ ]:
pst_rw = "swatp_rw.pst"

In [ ]:
# Let's create a new control file with the number of iterations set to 30 and incorporate reweighted factors.
pst.control_data.noptmax = 0
pst.write(os.path.join(main_opt_path, f'{pst_rw}'), version=2)

In [ ]:
# pyemu.os_utils.run(f'pestpp-glm.exe {pst_rw}' , cwd=".")

# 03. Perform uncertainty analysis with IES

In [ ]:
# check the number of available cores
num_workers = psutil.cpu_count(logical=False)
num_workers

In [ ]:
main_opt_path

In [ ]:
pst_nam_ies = "swatp_rw_ies.pst"

In [ ]:
pst_ies = pyemu.Pst(os.path.join(main_opt_path, f"{pst_rw}"))
# set IES
pst_ies.pestpp_options['ies_num_reals'] = 50 # number of realization
pst_ies.control_data.noptmax = 5 # number of iteration
pst_ies.write(f'{pst_nam_ies}', version=2) # write new IES control file

In [ ]:
# set a path to the main directory of model
m_d = os.path.join(prj_dir, "swatp_rw_ies")

In [ ]:
m_d

In [ ]:
main_opt_path

In [ ]:
prj_dir

In [ ]:
os.chdir(prj_dir) # move the loction to project directory
# run PEST in parallel
pyemu.os_utils.start_workers(
                            main_opt_path, # the folder which contains the "template" PEST dataset
                            "pestpp-ies", #the PEST software version we want to run
                            f'{pst_nam_ies}', # the control file to use with PEST
                            num_workers=num_workers, #how many agents to deploy
                            worker_root='.', #where to deploy the agent directories; relative to where python is running
                            master_dir=m_d, #the manager directory,
                            # reuse_master=True
                            )

# 03. Let's see how we did:

In [ ]:
from swatp_pst import analyzer
import pyemu
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [ ]:
# m_d = 'D:\\spark\\Albufera\\opt01\\alb_rw_ies02'

In [ ]:
pst_file = pst_nam_ies
pst = pyemu.Pst(os.path.join(m_d, pst_file))

### 03.01 phi progress

In [ ]:
pst.phi

In [ ]:
# plot phi progress
# pyemu.plot_utils.phi_progress(pst)

In [ ]:
pst_file = pst_nam_ies
pst = pyemu.Pst(os.path.join(m_d, pst_file))

In [ ]:
pst_nam_ies[:-4]

In [ ]:
f"{pst_nam_ies[:-4]}.0.obs.csv"

In [ ]:
f"{pst_nam_ies[:-4]}{0}.obs.csv".format(4)

In [ ]:
# load prior simulation
pr_oe = pyemu.ObservationEnsemble.from_csv(
    pst=pst,filename=os.path.join(m_d,f"{pst_nam_ies[:-4]}.0.obs.csv")
    )
# load posterior simulation
pt_oe = pyemu.ObservationEnsemble.from_csv(
    pst=pst,
    filename=os.path.join(m_d,f"{pst_nam_ies[:-4]}.{4}.obs.csv")
    )

In [ ]:
pt_oe

## 03.02 Predictive uncertainty

In [ ]:
analyzer.single_plot_tseries_ensembles_plots_added(
    pst, pr_oe, pt_oe, width=10, height=5, dot=False,
    # bstcs=["56", "171"], 
    # orgsim=org_sim
)

## 03.03 Parameter uncertainty

In [ ]:
prior_df = pyemu.ParameterEnsemble.from_csv(
    pst=pst,filename=os.path.join(m_d,"swatp_rw_ies.{0}.par.csv".format(0)))
post_df = pyemu.ParameterEnsemble.from_csv(
    pst=pst,filename=os.path.join(m_d,"swatp_rw_ies.{0}.par.csv".format(4)))

In [ ]:
df_pars = pd.read_csv(os.path.join(m_d, "swatp_rw_ies.par_data.csv"))
sel_pars = df_pars.loc[df_pars["partrans"]=='log']
sel_pars

In [ ]:
analyzer.plot_prior_posterior_par_hist(m_d, pst, prior_df, post_df, sel_pars, height=7)

# 04 updates the model input files with parameter values

In [ ]:
# m_d = "D:\\jj\\opt_3rd\\calibrated"
# pst_file = "alb_rw_ies.pst"
# pst = pyemu.Pst(os.path.join(m_d, pst_file))

In [ ]:
# pst.parrep(parfile=os.path.join(m_d, "alb_rw_ies.{0}.base.par".format("final")))
# # updates the model input files with parameter values
# pst.write_input_files(pst_path=m_d)
# # run the model forward run; this applies all the SWAT+ paarameters, executes SWAT+ 
# os.chdir(m_d)
# pyemu.os_utils.run('python forward_run.py')

In [ ]:
# pyemu.os_utils.run("python forward_run.py",cwd=m_d)